In [1]:
import sys
sys.path.append('C:\Program Files\Anaconda3\Lib\site-packages')

In [2]:
import json, os
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [3]:
pd.set_option('display.max_colwidth', 1000)

## Данные

Возьмем данные вот отсюда - https://github.com/mannefedov/ru_kw_eval_datasets Там лежат 4 датасета (статьи с хабра, с Russia Today, Независимой газеты и научные статьи с Киберленинки). Датасет НГ самый маленький, поэтому возьмем его в качестве примера.

In [4]:
# скачаем данные в папке data и распакуем их
PATH_TO_DATA = './data'

In [5]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [6]:
files = [file.replace('\\','/') for file in files]

In [7]:
files

['./data/ng_0.jsonlines', './data/ng_1.jsonlines']

Объединим файлы в один датасет.

In [8]:
data = pd.concat([pd.read_json(file, lines=True, encoding = 'utf-8') for file in files], axis=0, ignore_index=True)

In [9]:
data.shape

(1987, 5)

In [10]:
data.head(3)

,content,keywords,summary,title,url
0,"Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам боль...","[яблоко, молодежь, молодежное яблоко]",,"""Молодежное ""Яблоко"": оппозиционная деятельность становится опасной",http://www.ng.ru/ng_politics/2017-04-18/11_6976_apple.html
1,"Вчера «Газпром» снизил верхнюю планку прогноза собственной добычи газа в 2020 году. Через 12 лет концерн собирается добывать около 620–640 млрд. куб. м в год. При этом общее производство газа в стране, по расчетам холдинга, должно достичь 940 млрд. куб. м. Иными словами, треть добываемого объема, по мнению холдинга, должны будут обеспечить независимые производители. Эксперты не верят, что независимые компании смогут выйти на такие объемы добычи. Если расчеты «Газпрома» не оправдаются, то под ударом окажутся отечественные предприятия и население, которым придется сокращать потребление и смириться с новым витком цен. Иных путей покрытия возможного дефицита газа нет, так как вряд ли холдинг разорвет уже заключенные контракты на экспорт газа в другие страны. \n«Газпром» к 2020 году планирует добывать 620–640 млрд. куб. м газа, сообщил вчера на форуме «ТЭК России в ХХI веке» глава управления по добыче газа, газового конденсата и нефти холдинга Валерий Минликаев. Тем самым он уточнил пре...","[газпром, газ]",,"""Газпрома"" на всех не хватит",http://www.ng.ru/economics/2008-04-03/1_gazprom.html
2,"Долголетний труд Евгения Витковского на ниве перевода, а также в качестве редактора и антологиста известен многим. Но не все знают его как поэта и прозаика. В этом году уже вышла составленная им и Еленой Кистеровой антология «Раздол туманов: Страницы шотландской гэльской поэзии XVII–XX вв.», а в апреле запланирован выход его романа «Протей, или Византийский кризис» (отрывок из романа читайте на с. 12). С \n побеседовал \n– Одна из таких книг только что вышла – «Раздол туманов. Страницы шотландской гэльской поэзии XVII–XX веков». Это стихи 29 поэтов, все в переводе с оригинала – моем и Елены Кистеровой. Работа заняла 10 лет, включая изучение языка. Она была упоительно интересной: до нас переводов из этой поэзии на русский не было вовсе. Сейчас должен выйти том стихотворений канадского классика Роберта Уильяма Сервиса, «канадского Киплинга», около 300 стихотворений. Кроме того, в Петербурге в производстве наш огромный трехтомный плод совместной работы – антология «Франция в сердце»....","[франсуа рабле, сервантес, шекспир, конан дойл, михаил булгаков, александр грин, борхес, босх, маркес, герман гессе, голландская живопись, гаргантюа и пантагрюэль, дон кихот, мастер и маргарита, москва, россия, история, поэзия, шотландия, баллада, пере]","Евгений Витковский о том, как Босх протягивает руку Шекспиру, \r\nи оба танцуют в пламени пожара в охваченном чумой средневековом городе",Бесконечная партия в четырехмерные шахматы,http://www.ng.ru/person/2018-03-22/10_927_vitkovsky.html


Каждой статье приписано какое-то количество ключевых слов. Допустим, что это единственно правильный набор ключевых слов (что конечно не так, но других данных у нас нет). Наша задача - придумать как извлекать точно такой же список автоматически.  
Зададим несколько метрик, по которым будем определять качество извлекаемых ключевых слов - точность, полноту, ф1-меру и меру жаккарда.

In [11]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))
    
    
        

Проверим, что всё работает как надо.

In [4]:
#data = pd.read_csv('data.csv', encoding = 'utf-8')

In [12]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


# Тупое решение.

Давайте не будем думать, а попробуем сразу придумать какое-то решение.

Возьмем первые 5 слов из заголовка.

In [13]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:5]))

Precision -  0.06
Recall -  0.05
F1 -  0.05
Jaccard -  0.03


Или 10.

In [14]:
evaluate(data['keywords'], data['title'].apply(lambda x: x.lower().split()[:10]))

Precision -  0.06
Recall -  0.06
F1 -  0.05
Jaccard -  0.03


Теперь попробуем взять самые частотные слова.

In [15]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 [x[0] for x in Counter(x.lower().split()).most_common(10)]))

Precision -  0.02
Recall -  0.04
F1 -  0.02
Jaccard -  0.01


Или вообще рандомные слова.

In [17]:
evaluate(data['keywords'], data['content'].apply(lambda x: 
                                                 np.random.choice(list(set(x.lower().split())), 10)))

Precision -  0.01
Recall -  0.01
F1 -  0.01
Jaccard -  0.0


Теперь давайте посмотрим, что вообще извлекается.

In [18]:
data['title'].apply(lambda x: x.lower().split()[:10]).head(10)

0                         ["молодежное, "яблоко":, оппозиционная, деятельность, становится, опасной]
1                                                                 ["газпрома", на, всех, не, хватит]
2                                                   [бесконечная, партия, в, четырехмерные, шахматы]
3    [экс-депутат,, осужденная, за, фальсификацию, выборов,, оказалась, членом, "боевого, братства"]
4                               [новая, москва, останется, территорией, экологической, безопасности]
5                                [f1., гран-при, сша, прошел, без, четырех, машин, и, со, «стопкой»]
6                                          [100, ведущих, политиков, россии, в, феврале, 2018, года]
7                                               [закон, "о, культуре", принимают, на, фоне, арестов]
8                                    [насколько, реальна, газовая, подоплека, сирийского, конфликта]
9                                  [фсб:, в, калужской, области, задержаны, четверо, участн

In [19]:
data['content'].apply(lambda x: [x[0] for x in Counter(x.lower().split()).most_common(10)]).head(10)

0                                                    [в, и, на, что, не, –, его, а, это, из]
1                                                 [в, и, на, –, куб., млрд., по, к, газа, м]
2                                                      [в, –, и, не, я, но, это, что, на, с]
3                                                 [в, на, и, ким, –, по, что, он, видео, до]
4                                   [в, и, на, площадью, новой, москвы, –, развития, с, га.]
5                                                  [в, на, и, не, с, но, уже, что, гонки, у]
6                                       [на, в, (с, место)., и, рф, влияние, позиции, по, с]
7                                             [в, и, –, по, с, как, будет, культуре, из, не]
8                                                   [в, и, на, с, что, для, по, –, не, газа]
9    [в, террористической, организации, задержаны, рф, –, боевую, цос,, исламистов, жителей]
Name: content, dtype: object

Когда извлекаются частотные слова, то список почти полностью состоит из всяких стоп-слов. Также из-за плохой токенизации некоторые слова в обоих списках - пунктуация или слова с пунктуацией на концах. К тому же извлекаемые слова ненормализованы, а правильные ключевые слова - наоборот.

## Токенизация, удаление стоп-слов и нормализация.

In [20]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

import re
from stop_words import get_stop_words
stopwords = get_stop_words('russian')

def normalize(text):
    
    #words = [word.strip(punct) for word in text.lower().split()]
    words = re.findall(r'[a-zа-яё]+', text.lower())
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stopwords]

    return words

In [ ]:
Изменим немного функцию предобработки, чтобы оставить только буквенные символы.

In [22]:
data['content_norm'] = data['content'].apply(normalize)

In [58]:
%%time
data['title_norm'] = data['title'].apply(normalize)

Wall time: 6.95 s


In [59]:
data['title_norm'].head(10)

0             [молодёжный, яблоко, оппозиционный, деятельность, становиться, опасный]
1                                                                  [газпром, хватить]
2                                       [бесконечный, партия, четырехмерный, шахматы]
3    [экс, депутат, осудить, фальсификация, выбор, оказаться, член, боевой, братство]
4                  [новый, москва, остаться, территория, экологический, безопасность]
5                               [f, гран, сша, пройти, четыре, машина, стопка, штраф]
6                                                 [ведущий, политик, россия, февраль]
7                                            [закон, культура, принимать, фон, арест]
8                      [насколько, реальный, газовый, подоплёка, сирийский, конфликт]
9                        [фсб, калужский, область, задержать, четверо, участник, иго]
Name: title_norm, dtype: object

Попробуем те же самые методы.

In [60]:
# топ 10 частотных слов статьи
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


In [61]:
evaluate(data['keywords'],data['title_norm'].apply(lambda x: x[:10]))

Precision -  0.14
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


In [39]:
evaluate(data['keywords'], data['content'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.92
Recall -  0.39
F1 -  0.55
Jaccard -  0.38


Качество сильно улучшилось! Можно теперь ещё раз посмотреть, что плохого извлекается.

In [62]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(20)

0        [яблоко, молодёжный, акция, активист, деятельность, политика, московский, власть, уровень, проводить]
1              [миллиард, газа, куб, газпром, добыча, производитель, страна, независимый, внутренний, должный]
2                                [книга, роман, писать, выйти, век, стихотворение, перевод, жанр, мир, издать]
3                          [ким, зинаида, видео, журналист, бывший, рубль, судебный, заседание, дело, футиный]
4                              [площадь, москва, га, новый, территория, тинао, столица, развитие, парка, парк]
5                               [гонка, позиция, команда, место, круг, чемпионат, два, бокс, пилот, следующий]
6                            [место, го, влияние, рф, позиция, глава, политический, россия, президент, сергей]
7              [культура, закон, стд, сфера, общественный, услуга, проект, учреждение, сообщество, предложить]
8                        [газопровод, сирия, турция, турецкий, газа, европа, катар, россия, российский, поток]
9

Ещё остались некоторые стоп-слова. Вместо того, чтобы расширять список, давайте попробуем выкинуть несуществительные.

In [25]:
def normalize(text):
    
#     words = [word.strip(punct) for word in text.lower().split()]
#     words = [morph.parse(word)[0] for word in words if word and word not in stops]
    
    words = re.findall(r'[a-zа-яё]+', text.lower())
   
    parsed = [morph.parse(word)[0] for word in words if word and word not in stopwords and len(word)>2]
    words = [parsed_word.normal_form for parsed_word in parsed if parsed_word.tag.POS == 'NOUN']

    return words

In [28]:
%%time
data['content_norm'] = data['content'].apply(normalize)

Wall time: 9min 13s


In [29]:
data.to_csv('data_norm.csv', encoding = 'utf-8', index=False)

In [31]:
evaluate(data['keywords'], data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Ещу улучшения!

In [32]:
data['content_norm'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]).head(10)

0                     [яблоко, акция, активист, деятельность, политика, власть, россия, задача, дарья, жизнь]
1                     [миллиард, газа, куб, газпром, добыча, производитель, страна, прогноз, холдинг, расчёт]
2                             [книга, роман, жанр, стихотворение, мир, читатель, перевод, россия, герой, том]
3                          [ким, зинаида, видео, журналист, процесс, дело, рубль, экспертиза, суд, свидетель]
4                        [площадь, москва, территория, парк, развитие, парка, тинао, столица, регион, власть]
5                                [гонка, команда, позиция, место, круг, пилот, бокс, чемпионат, баттон, балл]
6                     [место, влияние, позиция, глава, президент, сергей, россия, участие, рейтинг, институт]
7             [культура, закон, сфера, учреждение, сообщество, услуга, проект, концепция, изменение, человек]
8                               [газопровод, сирия, турция, газа, катар, европа, россия, поток, иран, проект]
9    [учас

Не очень значимые слова все ещё остались. Давайте попробуем отсеять стоп-слова с помощью tfidf.

Воспользуемся TfidfVectorizer.

In [33]:
data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [34]:
# можно заодно сделать нграммы
tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=5)

In [35]:
tfidf.fit(data['content_norm_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [36]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}

Преобразуем наши тексты в векторы, где на позиции i стоит tfidf коэффициент слова i из словаря.

In [37]:
texts_vectors = tfidf.transform(data['content_norm_str'])

Отсортируем векторы текстов по этим коэффициентам и возьмем топ-10.

In [38]:
# сортировка по убыванию, поэтому нужно развернуть список
keywords = [[id2word[w] for w in top] for top in texts_vectors.toarray().argsort()[:,:-11:-1]] 

In [39]:
keywords[:3]

[['яблоко',
  'активист',
  'акция',
  'дарья',
  'деятельность',
  'политика',
  'виктор',
  'тимур',
  'репрессия',
  'выборы'],
 ['миллиард куб',
  'куб',
  'газпром',
  'газа',
  'миллиард',
  'добыча',
  'добыча газа',
  'холдинг',
  'производитель',
  'прогноз'],
 ['роман',
  'книга',
  'жанр',
  'стихотворение',
  'читатель',
  'том',
  'перевод',
  'поэзия',
  'произведение',
  'туманов']]

In [40]:
evaluate(data['keywords'], keywords)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


Результат ещё немного улучшился. Немного подросла точность. Теперь вместо стоп-слов в ключевые попадают имена и все такое. Иногда это хорощо, а иногда нет (собянин - может быть ключевым словом, а дарья - вряд ли)

Возьмем этот результат за baseline. 

Precision -  0.13
Recall -  0.24
F1 -  0.16
Jaccard -  0.09

## Попробуем графы!

Большая часть методов для извлечения ключевых слов основана на применении графов. Основная идея - каким-то образом перевести текст в граф, а затем каким-то образом расчитать важность каждого узла и вывести топ-N самых важных узлов.  

Перевод текста в граф -  не тривиальная задача. Часто применяют такой подход - построим матрицу совстречаемости слов (в каком-то окне), эта матрица будет нашей матрицей смежности.

Для выбора важных узлов часто используют простой randow walk. Алгоритм примерно такой:  
1) Каким-то образом выбирается первый узел графа (например, случайно из равномерного распределения)  
2) на основе связей этого узла с другими, выбирается следующий узел  
3) шаг два повторяется некоторое количество раз (например, тысячу) __*чтобы не зацикливаться, с какой-то вероятностью мы случайно перескакиваем на другой узел (даже если он никак не связан с текущим, как в шаге 1)__  
5) на каждом шаге мы сохраняем узел в котором находимся  
6) в конце мы считаем в каких узлах мы были чаще всего и выводим top-N  


Предполагается, что мы часто будем приходить в важные узлы графа.

In [41]:
from itertools import combinations

Для наглядности реализуем этот подход без networkx. 

In [42]:
def get_kws(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    # нормализуем строки, чтобы получилась вероятность перехода
    for i in range(m.shape[0]):
        m[i] /= np.sum(m[i])
    
    # случайно выберем первое слова, а затем будет выбирать на основе полученых распределений
    # сделаем так 5 раз и добавим каждое слово в счетчик
    # чтобы не забиться в одном круге, иногда будет перескакивать на случайное слово
    
    c = Counter()
    # начнем с абсолютного случайно выбранного элемента
    n = np.random.choice(len(vocab))
    for i in range(500): # если долго считается, можно уменьшить число проходов
        
        # c вероятностью random_p 
        # перескакиваем на другой узел
        go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
        if go_random:
            n = np.random.choice(len(vocab))
        
        n = take_step(n, m)
        # записываем узлы, в которых были
        c.update([n])
    
    # вернем топ-N наиболее часто встретившихся сл
    return [id2word[i] for i, count in c.most_common(top)]

def take_step(n, matrix):
    rang = len(matrix[n])
    # выбираем узел из заданного интервала, на основе распределения из матрицы совстречаемости
    next_n = np.random.choice(range(rang), p=matrix[n])
    return next_n
    


In [43]:
%%time
keywords_rw = data['content_norm'].apply(lambda x: get_kws(x, 10, 10))

Wall time: 3min 50s


In [44]:
evaluate(data['keywords'], keywords_rw)

Precision -  0.11
Recall -  0.22
F1 -  0.14
Jaccard -  0.08


In [45]:
keywords_rw.head(10)

0            [яблоко, активист, россия, женщина, власть, виктор, сторона, акция, выборы, искренность]
1       [газпром, миллиард, куб, газа, добыча, страна, производитель, прогноз, поставка, потребление]
2                         [роман, книга, жанр, мир, мениппея, герой, павел, поэзия, история, прозаик]
3             [ким, зинаида, свидетель, рубль, видео, тысяча, экспертиза, прошение, работа, защитник]
4                       [площадь, парка, москва, территория, столица, парк, регион, вид, тинао, зона]
5                         [гонка, команда, место, баттон, круг, позиция, больница, сша, заезд, перес]
6             [место, позиция, влияние, сообщение, глава, сергей, участие, президент, политолог, фон]
7          [культура, закон, сфера, учёт, сообщество, учреждение, человек, вопрос, взгляд, изменение]
8                     [газопровод, катар, газа, поток, турция, сирия, европа, поставка, россия, иран]
9    [участник, область, организация, ячейка, центр, государство, подготовка, март

Попбруем теперь важность считать с помощью какой-нибудь метрики из networkx.

In [46]:
import networkx as nx

In [162]:
def build_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            # чтобы граф был ненаправленный 
            m[j][k] += 1
            m[k][j] += 1
    
    return m, id2word

def some_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_matrix(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.degree(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [48]:
%%time
keyword_nx = data['content_norm'].apply(lambda x: some_centrality_measure(x, 10, 10))

Wall time: 45 s


In [49]:
evaluate(data['keywords'], keyword_nx)

Precision -  0.13
Recall -  0.25
F1 -  0.16
Jaccard -  0.09


In [51]:
keyword_nx.head(10)

0                [яблоко, акция, активист, деятельность, политика, власть, дарья, жизнь, россия, задача]
1                [газа, газпром, миллиард, куб, добыча, производитель, холдинг, прогноз, расчёт, страна]
2                     [книга, роман, читатель, жанр, мир, стихотворение, перевод, мениппея, работа, том]
3                [ким, зинаида, видео, процесс, свидетель, рубль, экспертиза, заседание, журналист, суд]
4              [москва, площадь, территория, столица, тинао, регион, развитие, парка, парк, департамент]
5                           [гонка, позиция, команда, место, круг, чемпионат, пилот, заезд, льюис, бокс]
6                     [место, влияние, позиция, глава, россия, президент, сергей, участие, рейтинг, фон]
7         [культура, закон, сфера, учреждение, сообщество, концепция, изменение, дело, задача, характер]
8                          [сирия, газопровод, катар, газа, европа, россия, турция, поток, иран, проект]
9    [участник, территория, подготовка, прошедшее, жите

1. Попробуем другие метрики вершин графа, например, closeness_centrality, характеризующую центральность вершины в терминах расстояний до остальных вершин (геометрический центр)

In [75]:
def close_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_matrix(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.closeness_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [79]:
keywords_between = close_centrality_measure(data.content_norm[0], 7, 10)

In [81]:
%%time
keywords_between = data['content_norm'].apply(lambda x: close_centrality_measure(x, 7, 10))

Wall time: 5min 3s


In [82]:
evaluate(data['keywords'], keywords_between)

Precision -  0.12
Recall -  0.24
F1 -  0.15
Jaccard -  0.09


In [126]:
matrix, id2word = build_matrix(data.content_norm[0], 10)

In [127]:
G = nx.from_numpy_matrix(matrix)

In [128]:
G.remove_edges_from(G.selfloop_edges())

In [113]:
sorted(nx.core_number(G).items(), key = lambda x: x[1], reverse=True)

[(0, 9),
 (1, 9),
 (2, 9),
 (4, 9),
 (5, 9),
 (6, 9),
 (7, 9),
 (8, 9),
 (9, 9),
 (10, 9),
 (11, 9),
 (12, 9),
 (13, 9),
 (14, 9),
 (16, 9),
 (17, 9),
 (18, 9),
 (19, 9),
 (20, 9),
 (21, 9),
 (22, 9),
 (24, 9),
 (26, 9),
 (27, 9),
 (28, 9),
 (29, 9),
 (30, 9),
 (31, 9),
 (32, 9),
 (33, 9),
 (34, 9),
 (35, 9),
 (38, 9),
 (39, 9),
 (40, 9),
 (41, 9),
 (42, 9),
 (43, 9),
 (44, 9),
 (45, 9),
 (46, 9),
 (47, 9),
 (48, 9),
 (49, 9),
 (50, 9),
 (51, 9),
 (52, 9),
 (53, 9),
 (54, 9),
 (55, 9),
 (56, 9),
 (57, 9),
 (58, 9),
 (59, 9),
 (60, 9),
 (61, 9),
 (63, 9),
 (64, 9),
 (65, 9),
 (66, 9),
 (67, 9),
 (68, 9),
 (69, 9),
 (70, 9),
 (71, 9),
 (72, 9),
 (73, 9),
 (74, 9),
 (75, 9),
 (76, 9),
 (77, 9),
 (78, 9),
 (79, 9),
 (81, 9),
 (82, 9),
 (83, 9),
 (84, 9),
 (86, 9),
 (87, 9),
 (88, 9),
 (89, 9),
 (90, 9),
 (91, 9),
 (92, 9),
 (96, 9),
 (97, 9),
 (98, 9),
 (99, 9),
 (100, 9),
 (101, 9),
 (102, 9),
 (103, 9),
 (104, 9),
 (105, 9),
 (106, 9),
 (108, 9),
 (109, 9),
 (110, 9),
 (111, 9),
 (112, 9

Попробуем выделять самые важные вершины с точки зрения значения k-core

In [120]:
def get_k_core(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_matrix(matrix)
    G.remove_edges_from(G.selfloop_edges())
    
    node2measure = nx.core_number(G)
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [121]:
kewords_core = get_k_core(data.content_norm[0], 10, 10)

In [122]:
kewords_core

['люба',
 'обязанность',
 'месяц',
 'факт',
 'дело',
 'вопрос',
 'военный',
 'руководство',
 'смена',
 'сми']

In [124]:
%%time
kewords_core = data['content_norm'].apply(lambda x: get_k_core(x, 10, 10))

Wall time: 1min 3s


In [125]:
evaluate(data['keywords'], kewords_core)

Precision -  0.03
Recall -  0.06
F1 -  0.04
Jaccard -  0.02


Попробуем представить тексты в виде ориентированного графа и рассмтотреть входящую меру центральности слов.

In [138]:
def oriented_matrix(text, window_size=5):
    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    # преобразуем слова в индексы для удобства
    ids = [word2id[word] for word in text]

    # создадим матрицу совстречаемости
    m = np.zeros((len(vocab), len(vocab)))

    # пройдемся окном по всему тексту
    for i in range(0, len(ids), window_size):
        window = ids[i:i+window_size]
        # добавим единичку всем парам слов в этом окне
        for j, k in combinations(window, 2):
            #print(j, k)
            m[j][k] += 1
    
    return m, id2word

In [145]:
def in_degree_measure(text, window_size=5, topn=5):
    
    matrix, id2word = oriented_matrix(text, window_size)
    G = nx.DiGraph(matrix)
    node2measure = nx.in_degree_centrality(G)
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [146]:
in_degree_measure(data.content_norm[0], 10, 10)

['яблоко',
 'акция',
 'активист',
 'дарья',
 'деятельность',
 'политика',
 'власть',
 'выборы',
 'убийство',
 'март']

In [152]:
%%time
kewords_in = data['content_norm'].apply(lambda x: in_degree_measure(x, 10, 10))

Wall time: 30.9 s


In [153]:
evaluate(data['keywords'], kewords_in)

Precision -  0.12
Recall -  0.23
F1 -  0.15
Jaccard -  0.09


Попробуем удалить имена при нормализации

In [157]:
def normalize(text):
    
#     words = [word.strip(punct) for word in text.lower().split()]
#     words = [morph.parse(word)[0] for word in words if word and word not in stops]
    
    words = re.findall(r'[a-zа-яё]+', text.lower())
   
    parsed = [morph.parse(word)[0] for word in words if word and word not in stopwords and len(word)>2]
    words = [parsed_word.normal_form for parsed_word in parsed if parsed_word.tag.POS == 'NOUN' and \
             'Name' not in parsed_word.tag]

    return words

In [158]:
%%time
data['content_no_name'] = data['content'].apply(normalize)

Wall time: 17min 51s


In [171]:
%%time
keyword_noname = data['content_no_name'].apply(lambda x: some_centrality_measure(x, 10, 7))

Wall time: 52.3 s


In [172]:
evaluate(data['keywords'], keyword_noname)

Precision -  0.16
Recall -  0.21
F1 -  0.17
Jaccard -  0.1


In [173]:
keyword_noname.head(10)

0            [яблоко, акция, политика, активист, деятельность, власть, жизнь]
1              [газа, газпром, миллиард, куб, добыча, производитель, холдинг]
2                         [книга, жанр, роман, мир, герой, читатель, перевод]
3             [видео, журналист, дело, экспертиза, суд, свидетель, заседание]
4             [площадь, москва, территория, столица, регион, тинао, развитие]
5                    [гонка, позиция, команда, место, чемпионат, круг, пилот]
6                [место, влияние, позиция, глава, россия, президент, участие]
7      [культура, закон, сфера, сообщество, учреждение, концепция, изменение]
8                    [газопровод, сирия, россия, турция, катар, газа, европа]
9    [участник, территория, подготовка, прошедшее, житель, исламист, эмиссар]
Name: content_no_name, dtype: object

Небольшое изменение предобработки немного повысило f-меру

In [170]:
data['keywords'].apply(lambda x: len(x)).mean()

6.1177654755913435

Попробуем так же метрику betweenness_centrality, характеризующую степень того, насколько часто через вершину проходят кратчайшие пути между другими вершинами графа (слова, которые часто встречаются в разных контекстах).

In [174]:
def between_centrality_measure(text, window_size=5, topn=5):
    
    matrix, id2word = build_matrix(text, window_size)
    G = nx.from_numpy_matrix(matrix)
    # тут можно поставить любую метрику
    node2measure = dict(nx.betweenness_centrality(G))
    
    return [id2word[index] for index,measure in sorted(node2measure.items(), key=lambda x: -x[1])[:topn]]

In [176]:
from tqdm import tqdm_notebook

In [177]:
%%time
keyword_betw = [between_centrality_measure(x, 10, 7) for x in tqdm_notebook(data['content_no_name'])]


Wall time: 22min 45s


In [178]:
evaluate(data['keywords'], keyword_betw)

Precision -  0.15
Recall -  0.21
F1 -  0.16
Jaccard -  0.1


## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/Zb89yjXFr37EITMq1

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другой.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 0.5 баллов).